# Basic imports

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os, time
import requests, zipfile, io

# Import packages for machine learning analysis

In [ ]:
# Data representation, linear algebra
import numpy as np
import pandas as pd

# Machine learning
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.calibration import calibration_curve

# Set the transformer setting
from sklearn import set_config
set_config(transform_output = "pandas")

# Plotting
import matplotlib.pyplot as plt
plt.rc('font', size=18)
plt.rc('figure', figsize=(7,5))

# Load and inspect the data

In [ ]:
df = ...

## Inspect the dataframe

### Covariates
Variable name | Meaning
 --- | ---
native-country | Native country (categorical)
sex | Self-reported sex (binary)
race | Self-reported race (categorical)
age | Age (continuous)
education | Education level (categorical)
workclass | Class of job (categorical)
occupation | Type of job (categorical)
marital-status | Marital status (categorical)
relationship | Type of relationship / household (categorical) 
capital-net | Yearly change in capital (continuous)
hours-per-week | Number of work hours per week (continuous)
income_current | Current yearly income in USD (continuous)
education-num | Numeric interpretation of education (continuous)

### Treatment
Variable name | Meaning
 --- | ---
studies | Type of studies (categorical)

### Outcome
Variable name | Meaning
 --- | ---
income | Yearly income in USD **after 10 years**


### What are the types of studies?

In [ ]:
# The treatment variable
t_col = ...

plt.hist ... 
plt.xticks(rotation=90)

<br/><br/>
# Let's figure out our goal [first... back to the slides]
<br/><br/>

# Let's define our outcome variable and adjustment set

In [ ]:
# The outcome column
y_col = ... 

# The adjustment set
a_cols = ... 

# All covariates
x_cols = ... 

In [ ]:
# We only want to study 'Full-time studies' vs 'No studies'
# Let's restrict our dataframe to these

df = ...

# Regression adjustment requires assumptions

Exchangeability and Consistency can't be checked. But Treatment overlap can! Let's check it!

In [ ]:
# Treatment indicator
T = ...

# Visualize variables
for c in a_cols: ...
    plt...

...
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>
# Take-aways from overlap check?

## First, let's restrict our study population

In [ ]:
df = ...

## Second, let's restrict our adjustment set

In [ ]:
a_cols = ...

# OK, we know...
* Our target parameter (ATE)
* Our study population
* Our adjustment set
* Our statistical estimand
* Something about treatment overlap

## Let's do some estimation!

## For reference, let's create a Difference-in-means (DIM) estimate of ATE

In [ ]:
T = ...
ATE_dim = ...

<br/><br/>
# Next, let's do a regression-based estimate

In [ ]:
# Let's hold out 20% of data for testing/estimating effects using train_test_split
df_tr, df_te = ...

# Training samples
A_tr, Y_tr, T_tr = ...

# Test samples
A_te, Y_te, T_te = ...

# We need to standardize and dichotomize features
# We can use the "SubsetTransformer" which is part of IncomeSim/util.py
tf = ...

# Lets store the post-transformation columns
columns_tf = ...

# And have a look at the transformed data
...

## Let's try fitting a T-learner with Ridge regression

A T-learner estimates the potential outcomes under treatment and control separately. Under conditional exchangeability (ignorability) w.r.t. $X$, we have

$$
\mu_t(x) = \mathbb{E}[Y(t) \mid X=x] = \mathbb{E}[Y \mid T=t, X=x]
$$

We can estimate $\mu_t$ for $t \in \{0,1\}$ separately as two regressions

$$
\hat{\mu}_t(x) = \mbox{arg} \min_f \frac{1}{n_t}\sum_{i : t_i=t} (f(x_i) - y_i)^2
$$

The minimizer of the mean-squared error (MSE) is the conditional expectation. 

Then, we can estimate the conditional average treatment effect (CATE)
$$
\hat{\tau}(x) = \hat{\mu}_1(x) - \hat{\mu}_0(x)
$$
and the average treatment effect (ATE)
$$
\hat{\tau} =  \frac{1}{n}\sum_{i=1}^n \hat{\tau}(x_i) = \frac{1}{n}\sum_{i=1}^n (\hat{\mu}_1(x_i) - \hat{\mu}_0(x_i))
$$

In [ ]:
# Let's use scikit-learns grid search
param_grid = ... 
mu0_r = ...
mu1_r = ...

# Compute the R2 score
R20_r = ...

In [ ]:
# Estimate CATE and ATE
CATE_r = ...
ATE_r = ...

<br/><br/><br/>
## Are these results good enough? What about a different model?

In [ ]:
# Let's use scikit-learns grid search
param_grid = ... 
mu0_rf = ...
mu1_rf = ...

# Compute the R2 score
R20_rf = ...

In [ ]:
# Estimate CATE and ATE
CATE_rf = ...
ATE_rf = ...

<br/><br/><br/>
## Are these results good enough? What about a different adjustment set?

In [ ]:
a_cols_ext = ...
A_tr_ext = ...
A_te_ext = ...

# Repeat what we did before

<br/><br/><br/>
## What happens to our results?
* How do the R2 compare?
* How do the ATE estimates?

<br/><br/><br/>
## What would happen if we accidentally included "income_current"?

A so-called post-treatment variable

<br/><br/><br/>
# Finally, let's do a propensity-weighting estimate

Assume that $p(T=t \mid X=x) > 0$ for all $x, t$. Then, under conditional exchangeability w.r.t. $X$ and consistency, 

$$
\mu_1 = \mathbb{E}[Y(1)] = \mathbb{E}\left[Y \frac{p(T=t)}{p(T=t \mid X)} \mid T=t \right] = \mathbb{E}\left[Y \frac{p(T=t)}{e_t(X)} \mid T=t \right]
$$
where $e_1(x) = e(x) = p(T=1 \mid X=x)$ is the propensity score w.r.t. $x$ and $e_0(x) = 1-e(x)$.

We can easily estimate $p(T=t)$. This is just the maginal rate of treatment. 

$e(x) = p(T=1 \mid X=x)$ is a conditional probability of a binary variable. We can estimate this using a stochastic classifier such as logistic regression or random forests! Given an estimate $\hat{e}(x)$, we can estimate expected potential outcomes 

$$
\hat{\mu_t} = \frac{1}{n_t}\sum_{i : t_i = t}\frac{\hat{p}(T=t)}{\hat{e}_t(x_i)} y_i
$$
and the average treatment effect
$$
\hat{\tau} = \hat{\mu_1} - \hat{\mu_0}
$$

Let's do that!

In [ ]:

# Fit a propensity model, e.g., logistic regression
e1_lr = ...

# Evaluate propensity on training set and test set
e1_lr_tr = ...
e1_lr_te = ...

# Evaluate the propensity model
AUC_lr = ...

In [ ]:
# Might as well do a random forest as well
e1_rf = ...

# Evaluate propensity on training set and test set
e1_lr_tr = ...
e1_lr_te = ...

# Evaluate the propensity model
AUC_rf = ...

<br/><br/><br/>
## Let's assess calibration

In [ ]:
# Compute calibration curve
rate_lr, pred_lr = ...

plt. ...

<br/><br/><br/>
## Let's compute the IPW estimate

In [ ]:
# Marginal treatment probability
p1 = ...
# Importance weights for treated
ipw1 = ...
# Importance weights for control
ipw0 = ...

# Horvitz-Thompson estimator
ATE_ipwn = ...

<br/><br/><br/><br/><br/><br/>
## Does that look right?

### Let's try one more thing

In [ ]:
# Importance weights for treated
ipw1h = ...
# Importance weights for control
ipw0h = ...

# Hajek estimator (normalized weights)
ATE_ipwn = ...

# OK, we have several estimates...

In [ ]:
# ATE_dim
# ATE_r
# ATE_rf
# ATE_ipw
# ATE_ipwn

<br/><br/><br/>
# How do we know if they are any good?

To confirm ATE in the real world, we must run an experiment and estimate it. This will...
* Gathering a population
* Randomly assigning them to $T=0$ and $T=1$
* Observing outcomes and computing averages

## We can simulate that by setting a flag ```-p``` in the generator

In [ ]:
# Generate according to T=0 policy
%run generate.py -n 50000 -T 10 -p no

# Generate according to T=1 policy
%run generate.py -n 50000 -T 10 -p full

In [ ]:
# Load datasets
df1 = ...
df0 = ...

# Restrict populations in the same way to  20 <= Age <= 55
df1 = ...
df0 = ...

# On-policy ATE estimate
ATE_onp = ...

<br/><br/><br/>
# If we have time, we can also estimate CATE as a function of age

## Let's use our estimates from the Ridge and Random forest

In [ ]:
df_cate_r = ...